In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.4 MB/s eta 0:00:00


In [3]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00


In [4]:

from langchain.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI
from oakey import openai_key   # I've stored the key in another file (oakey.py)
import pandas as pd
import os

os.environ["OPENAI_API_KEY"] = openai_key


In [5]:
# Loading only the relevant files

d1= pd.read_csv('/fin-data/US_INDU_INCOME_STATEMENT.csv')
d2 = pd.read_csv('/fin-data/US_INDU_CALCULATIONS.csv')

In [6]:
# Identifying the common columns between the 2 dataframes

r = d2.columns.intersection(d1.columns)
r

Index(['fundamental_id', 'company_id', 'name', 'cik', 'ticker', 'start_date',
       'end_date', 'months', 'fiscal_year', 'fiscal_period', 'filing_date'],
      dtype='object')

In [7]:
# Selecting only the relevant / needed columns from the 2nd dataframe

d2 = d2[['fundamental_id', 'company_id', 'name', 'start_date', 'end_date',
       'fiscal_year', 'fiscal_period', 'pricetoearnings', 'dividendyield', 'roe']]

In [9]:
# Joining the dataframes into one (similar to SQL join)

d = pd.concat([d1, d2], axis=1, join='outer')

In [10]:
# Removing the duplicate columns after the join
''' Converting the axis of the dataframe (cols to rows) in order to apply the 'drop_duplicates' built-in function
 which is used to drop duplicate rows, and then converting the result back into original columns format '''

d = d.T.drop_duplicates().T
d

,fundamental_id,company_id,name,cik,ticker,start_date,end_date,months,fiscal_year,fiscal_period,...,netincometocommon,weightedavebasicsharesos,basiceps,weightedavedilutedsharesos,dilutedeps,cashdividendspershare,fundamental_id,pricetoearnings,dividendyield,roe
0,fun_OrxGvb,com_NX6GzO,Apple Inc,320193.0,AAPL,2022-06-26,2022-09-24,3,2022,Q4,...,20721000000.0,16215963000.0,1.28,16325819000.0,1.28,0.23,fun_JboENB,24.5667,0.005993,1.754593
1,fun_eoG7eq,com_NX6GzO,Apple Inc,320193.0,AAPL,2021-09-26,2022-06-25,9,2022,Q3YTD,...,79082000000.0,16162945000.0,4.89,16262203000.0,4.89,0.69,fun_59kpv1,23.0516,0.006302,1.628163
2,fun_7KVkWO,com_NX6GzO,Apple Inc,320193.0,AAPL,2021-06-27,2022-06-25,12,2022,Q3TTM,...,99633000000.0,16162945000.0,6.16,16262203000.0,6.16,0.91,fun_035OM2,23.0516,0.006302,1.628163
3,fun_jv3WeG,com_NX6GzO,Apple Inc,320193.0,AAPL,2022-03-27,2022-06-25,3,2022,Q3,...,19442000000.0,16162945000.0,1.2,16262203000.0,1.2,0.23,fun_VbxQe0,23.0516,0.006302,1.628163
4,fun_alqowe,com_NX6GzO,Apple Inc,320193.0,AAPL,2021-09-26,2022-03-26,6,2022,Q2YTD,...,59640000000.0,16278802000.0,3.66,16403316000.0,3.66,0.45,fun_95Lnd6,27.9892,0.005059,1.492711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,fun_59GD2w,com_NgPvzd,Walmart Inc,104169.0,WMT,2019-08-01,2020-07-31,12,2021,Q2TTM,...,17895000000.0,2832000000.0,6.31,2848000000.0,6.27,2.16,fun_LJRjp1,19.8785,0.017133,0.228993
495,fun_rJNnZk,com_NgPvzd,Walmart Inc,104169.0,WMT,2020-05-01,2020-07-31,3,2021,Q2,...,6476000000.0,2832000000.0,2.29,2848000000.0,2.27,0.0,fun_vxNZpa,19.8785,0.017133,0.248638
496,fun_Grr49b,com_NgPvzd,Walmart Inc,104169.0,WMT,2019-05-01,2020-04-30,12,2021,Q1TTM,...,15029000000.0,2831000000.0,5.29,2849000000.0,5.26,2.16,fun_xYYL2K,22.1438,0.018234,0.206268
497,fun_N8VEDM,com_NgPvzd,Walmart Inc,104169.0,WMT,2020-02-01,2020-04-30,3,2021,Q1,...,3990000000.0,2831000000.0,1.41,2849000000.0,1.4,2.16,fun_pRRDw1,22.1438,0.018234,0.211976


In [11]:
d = d[['fundamental_id', 'company_id', 'name', 'fiscal_year', 'totalrevenue' ,
       'pricetoearnings', 'netincometocommon', 'dividendyield', 'roe']]

In [12]:
# Checking if there any null values after the join

d.loc[:, d.isnull().all()]

""
0
1
2
3
4
...
494
495
496
497


In [17]:
# Renaming columns as per the keys

''' d.rename(columns = {'totalrevenue': 'Total Revenue',
'netincometocommon':'netincometocommon',
'pricetoearnings':'Price to Earnings (P/E)',
'dividendyield':'Dividend Yield',
'roe':'Return on Equity (ROE)'}, inplace = True) '''

In [13]:
# The final version of the data before feeding the model
d

,fundamental_id,fundamental_id,company_id,name,fiscal_year,totalrevenue,pricetoearnings,netincometocommon,dividendyield,roe
0,fun_OrxGvb,fun_JboENB,com_NX6GzO,Apple Inc,2022,90146000000.0,24.5667,20721000000.0,0.005993,1.754593
1,fun_eoG7eq,fun_59kpv1,com_NX6GzO,Apple Inc,2022,304182000000.0,23.0516,79082000000.0,0.006302,1.628163
2,fun_7KVkWO,fun_035OM2,com_NX6GzO,Apple Inc,2022,387542000000.0,23.0516,99633000000.0,0.006302,1.628163
3,fun_jv3WeG,fun_VbxQe0,com_NX6GzO,Apple Inc,2022,82959000000.0,23.0516,19442000000.0,0.006302,1.628163
4,fun_alqowe,fun_95Lnd6,com_NX6GzO,Apple Inc,2022,221223000000.0,27.9892,59640000000.0,0.005059,1.492711
...,...,...,...,...,...,...,...,...,...,...
494,fun_59GD2w,fun_LJRjp1,com_NgPvzd,Walmart Inc,2021,542026000000.0,19.8785,17895000000.0,0.017133,0.228993
495,fun_rJNnZk,fun_vxNZpa,com_NgPvzd,Walmart Inc,2021,137742000000.0,19.8785,6476000000.0,0.017133,0.248638
496,fun_Grr49b,fun_xYYL2K,com_NgPvzd,Walmart Inc,2021,534661000000.0,22.1438,15029000000.0,0.018234,0.206268
497,fun_N8VEDM,fun_pRRDw1,com_NgPvzd,Walmart Inc,2021,134622000000.0,22.1438,3990000000.0,0.018234,0.211976


In [14]:
# Initializing the agent

agent = create_pandas_dataframe_agent(OpenAI(temperature=0), d, verbose=True)

In [16]:
# Checking companies names

d["name"].unique()

array(['Apple Inc', 'Amgen Inc', 'Boeing Co', 'Caterpillar Inc',
       'Salesforce.com Inc', 'Cisco Systems Inc', 'Chevron Corp',
       'Walt Disney Co, The', 'DOW INC.', 'Dow Inc.', 'Home Depot Inc',
       'Honeywell International Inc',
       'International Business Machines Corp', 'Intel Corp',
       'Johnson & Johnson', 'Coca-Cola Co', "McDonald's Corp", '3M Co',
       'Merck & Co Inc', 'Microsoft Corp', 'Nike Inc',
       'Procter & Gamble Co', 'UnitedHealth Group Inc', 'Visa Inc',
       'Verizon Communications Inc', 'Walgreens Boots Alliance Inc',
       'Walmart Inc'], dtype=object)

In [17]:
# Retrieving data from one company as a sample

d[d['name']=='Apple Inc']

,fundamental_id,fundamental_id,company_id,name,fiscal_year,totalrevenue,pricetoearnings,netincometocommon,dividendyield,roe
0,fun_OrxGvb,fun_JboENB,com_NX6GzO,Apple Inc,2022,90146000000.0,24.5667,20721000000.0,0.005993,1.754593
1,fun_eoG7eq,fun_59kpv1,com_NX6GzO,Apple Inc,2022,304182000000.0,23.0516,79082000000.0,0.006302,1.628163
2,fun_7KVkWO,fun_035OM2,com_NX6GzO,Apple Inc,2022,387542000000.0,23.0516,99633000000.0,0.006302,1.628163
3,fun_jv3WeG,fun_VbxQe0,com_NX6GzO,Apple Inc,2022,82959000000.0,23.0516,19442000000.0,0.006302,1.628163
4,fun_alqowe,fun_95Lnd6,com_NX6GzO,Apple Inc,2022,221223000000.0,27.9892,59640000000.0,0.005059,1.492711
5,fun_ZYWvr4,fun_4QBbrZ,com_NX6GzO,Apple Inc,2022,386017000000.0,27.9892,101935000000.0,0.005059,1.492711
6,fun_YnL6k2,fun_25l8Ok,com_NX6GzO,Apple Inc,2022,97278000000.0,27.9892,25010000000.0,0.005059,1.492711
7,fun_Q8EPW1,fun_VbGqon,com_NX6GzO,Apple Inc,2022,378323000000.0,28.7924,100555000000.0,0.004935,1.455673
8,fun_jvd9bV,fun_6GBl8p,com_NX6GzO,Apple Inc,2022,123945000000.0,28.7924,34630000000.0,0.004935,1.455673
9,fun_bWZqPb,fun_olOrwl,com_NX6GzO,Apple Inc,2022,394328000000.0,24.5667,99803000000.0,0.005993,1.754593


In [15]:
agent.run('How much did Intel Corp generate in sales in 2021?')



> Entering new AgentExecutor chain...


RateLimitError: ignored

In [31]:
agent.run('What was the bottom line for Intel Corp common shareholders in 2022?')



> Entering new AgentExecutor chain...
Thought: I need to find the net income to common shareholders for Intel Corp in 2022.
Action: python_repl_ast
Action Input: df[(df['name'] == 'Intel Corp') & (df['fiscal_year'] == 2022)]['netincometocommon'].sum()
Observation: 82041000000.0
Thought: I now know the final answer
Final Answer: The bottom line for Intel Corp common shareholders in 2022 was 8.2041e+10.

> Finished chain.


'The bottom line for Intel Corp common shareholders in 2022 was 8.2041e+10.'

In [35]:
agent.run('How much was verizon stock worth compared to its earnings in 2021?')



> Entering new AgentExecutor chain...


Thought: I need to find the price to earnings ratio for Verizon in 2021
Action: python_repl_ast
Action Input: df[(df['name'] == 'Verizon') & (df['fiscal_year'] == 2021)]['pricetoearnings'].values[0]
Observation: IndexError: index 0 is out of bounds for axis 0 with size 0
Thought:

 There is no data for Verizon in 2021
Action: python_repl_ast
Action Input: df[(df['name'] == 'Verizon') & (df['fiscal_year'] == 2020)]['pricetoearnings'].values[0]
Observation: IndexError: index 0 is out of bounds for axis 0 with size 0
Thought:

KeyboardInterrupt: ignored

In [125]:
agent1.run('What percentage of its share price did Visa return to its shareholders as dividends in 2022?')



> Entering new AgentExecutor chain...


KeyboardInterrupt: ignored

In [92]:
agent2.run('What was the efficiency of Apple in generating profits from its shareholders'' equity in 2021?')



> Entering new AgentExecutor chain...


InvalidRequestError: ignored